In [ ]:
import sys
if "pyodide" in sys.modules:
    import piplite
    await piplite.install('pyb2d-jupyterlite-backend>=0.4.2')


pyb2d is imported as b2d

In [ ]:
import b2d
# import pyb2d_jupyterlite_backend
from pyb2d_jupyterlite_backend.async_jupyter_gui import JupyterAsyncGui
import numpy as np
import matplotlib.pylab as plt

# Tutorial 0: A free falling body
The first step with Box2D is the creation of the world. The world is parametrized by a gravity vector.

In [ ]:
# the world
gravity = (0, -10)
world = b2d.World(gravity)

Create a circle-shaped body

In [ ]:
# the body def
body_def = b2d.BodyDef()
body_def.type = b2d.BodyType.dynamic
body_def.position = (0, 0)

# the body
body = world.create_body(body_def)

# shape
circle_shape = b2d.CircleShape()
circle_shape.radius = 1.0

# the fixture
fixture_def = b2d.FixtureDef()
fixture_def.shape = circle_shape
fixture_def.density = 1.0

# create and add the fixture to the body
fixture = body.create_fixture(fixture_def)

We can now have a look at the world: We render the world st. each meter in the Box2D world will be 100 pixels in the image:

In [ ]:
# from b2d.plot import render_world
b2d.plot.plot_world(world, ppm=100)

Lets run the world for a total of 5 seconds. 
Usually one wants to run the world at a certain frame rate.
With the frame rate and the total time we can compute the delta for each iteration and how many steps we need

In [ ]:
t = 5
fps = 40
dt = 1.0 / fps
n_steps = int(t / dt + 0.5)
print(f"t={t} fps={fps} dt={dt} n_steps={n_steps}")

in each step we query the bodies position and velocity and store then for later plotting

In [ ]:
positions = np.zeros([n_steps, 2])
velocites = np.zeros([n_steps, 2])
timepoints = np.zeros([n_steps])

t_elapsed = 0.0
for i in range(n_steps):

    # get the bodies center of mass
    positions[i, :] = body.world_center

    # get the bodies velocity
    velocites[i, :] = body.linear_velocity

    timepoints[i] = t_elapsed

    world.step(time_step=dt, velocity_iterations=1, position_iterations=1)
    t_elapsed += dt

plot the y-position against the time. We can see that the body is falling down in an accelerating way:

In [ ]:
plt.plot(timepoints, positions[:, 1])
plt.ylabel('y-poistion [meter]')
plt.xlabel('t [sec]')
plt.show()

as expected the x position is not changing since the gravity vector is non-zero only in the x direction

In [ ]:
plt.plot(timepoints, positions[:, 0])
plt.ylabel('x-poistion [meter]')
plt.xlabel('t [sec]')
plt.show()

# Tutorial 1: A  falling body in a box, more pythonic
Create a world, but in a more pythonic way, and animate the world

In [ ]:
# the world
world = b2d.world(gravity=(0, -10))

# create the dynamic body
body = world.create_dynamic_body(
    position=(5, 5),
    fixtures=b2d.fixture_def(shape=b2d.circle_shape(radius=1), density=1, restitution=0.75),
)

# create a box
box_shape = b2d.ChainShape()
box_shape.create_loop([(0, 0), (0, 10),(10,10),(10, 0)])
box = world.create_static_body(
    position=(0, 0), fixtures=b2d.fixture_def(shape=box_shape, friction=0)
)
b2d.plot.animate_world(world, ppm=20, t=10)

note that when we animate that world again, the body has already been fallen

In [ ]:
b2d.plot.animate_world(world, ppm=20, t=2)

# Tutorial 2: Interactive worlds
While animating the world already is already nice, interacting with the world is even better.
pyb2d has a framwork to interact with the world for multiple backends.
This framework is called `TestbedBase` since you can "test" your world in an interactive way

In [ ]:
from b2d.testbed import TestbedBase

class InteractiveExample(TestbedBase):
    def __init__(self, settings=None):
        super(InteractiveExample, self).__init__(settings=settings)
        # create two balls
        body = self.world.create_dynamic_body(position=(5, 5),
            fixtures=b2d.fixture_def(shape=b2d.circle_shape(radius=1), density=1, restitution=0.5),
        )
        body = self.world.create_dynamic_body(position=(8, 5),
            fixtures=b2d.fixture_def(shape=b2d.circle_shape(radius=1), density=1, restitution=0.8),
        )
        # create a box
        box_shape = b2d.ChainShape()
        box_shape.create_loop([(0, 0), (0, 10),(10,10),(10, 0)])
        box = self.world.create_static_body(
            position=(0, 0), fixtures=b2d.fixture_def(shape=box_shape, friction=0)
        )
        
s = JupyterAsyncGui.Settings()
s.resolution = [300,300]
b2d.testbed.run(InteractiveExample, backend=JupyterAsyncGui, gui_settings=s);

# Tutorial 3: Joints

## Tutorial 3.1: Prismatic Joint

In [ ]:
world = b2d.world(gravity=(0, -10))
anchor_body = world.create_static_body(position=(0, 0))
b = world.create_dynamic_body(
    position=(10, 10),
    fixtures=b2d.fixture_def(shape=b2d.polygon_shape(box=[2, 0.5]), density=1),
    linear_damping=0.0,
    angular_damping=0.0,
)
world.create_prismatic_joint(anchor_body, b, local_axis_a=(1, 1))
b2d.plot.animate_world(world, ppm=20, t=3, bounding_box=((0,0),(10,10)))

## Tutorial 3.2: Pully Joint

In [ ]:
world = b2d.world(gravity=(0, -10))


a = world.create_dynamic_body(
    position=(-5, 0),
    fixtures=b2d.fixture_def(shape=b2d.polygon_shape(box=[2, 0.8]), density=1),
    linear_damping=0.0,
    angular_damping=0.0,
)
b = world.create_dynamic_body(
    position=(5, 0),
    fixtures=b2d.fixture_def(shape=b2d.polygon_shape(box=[2, 0.5]), density=1),
    linear_damping=0.0,
    angular_damping=0.0,
)
world.create_pully_joint(
    a,
    b,
    length_a=10,
    length_b=10,
    ground_anchor_a=(-5, 10),
    ground_anchor_b=(5, 10),
    local_anchor_a=(0, 0),
    local_anchor_b=(0, 0),
)
b2d.plot.animate_world(world, ppm=20, t=5, bounding_box=((-10,-12),(10,12)))

## Tutorial 3.3: Revolute Joint

In [ ]:
world = b2d.world(gravity=(0, -10))
bodies = []
b = world.create_static_body(position=(0, 15))
bodies.append(b)
for i in range(5):
    b = world.create_dynamic_body(
        position=(i * 4 + 2, 15),
        fixtures=b2d.fixture_def(shape=b2d.polygon_shape(box=[2, 0.5]), density=1),
        linear_damping=0.0,
        angular_damping=0.0,
    )
    bodies.append(b)
world.create_revolute_joint(
    bodies[0], bodies[1], local_anchor_a=(0, 0), local_anchor_b=(-2, 0.0)
)
for i in range(1, len(bodies) - 1):
    a = bodies[i]
    b = bodies[i + 1]
    world.create_revolute_joint(a, b, local_anchor_a=(2, 0.0), local_anchor_b=(-2, 0.0))
b2d.plot.animate_world(world, ppm=20, t=5, bounding_box=((-20,-10),(20,20)))

## Tutorial 3.4: Weld Joint

In [ ]:
# the world
world = b2d.world(gravity=(0, -10))


bodies = []

# create  a static body as anchor
b = world.create_static_body(
    position=(0, 4), fixtures=b2d.fixture_def(shape=b2d.polygon_shape(box=[0.3, 0.5]))
)
bodies.append(b)

for i in range(4):
    b = world.create_dynamic_body(
        position=(i + 1.0, 4),
        fixtures=b2d.fixture_def(shape=b2d.polygon_shape(box=[0.3, 0.5]), density=0.1),
        linear_damping=2.5,
        angular_damping=2.5,
    )
    bodies.append(b)

for i in range(len(bodies) - 1):
    a = bodies[i]
    b = bodies[i + 1]
    world.create_weld_joint(
        a,
        b,
        local_anchor_a=(0.5, 0.5),
        local_anchor_b=(-0.5, 0.5),
        damping=0.1,
        reference_angle=0,
        stiffness=20,
    )
    world.create_weld_joint(
        a,
        b,
        local_anchor_a=(0.5, -0.5),
        local_anchor_b=(-0.5, -0.5),
        damping=0.1,
        reference_angle=0,
        stiffness=20,
    )
b2d.plot.animate_world(world, ppm=20, t=5, bounding_box=((0,-5),(5,5)))

## Tutorial 3.5: Wheel Joint

In [ ]:
world = b2d.world(gravity=(0, -10))
edge = world.create_static_body(
    position=(0, 0), fixtures=b2d.fixture_def(shape=b2d.edge_shape([(-20, 0), (5, 0)]))
)

# random slope
x = np.linspace(5, 50, 10)
y = np.random.rand(10) * 4 - 2
y[0] = 0
xy = np.stack([x, y]).T
xy = np.flip(xy, axis=0)
edge = world.create_static_body(
    position=(0, 0),
    fixtures=b2d.fixture_def(shape=b2d.chain_shape(xy, prev_vertex=(10, 0))),
)
# create car
left_wheel = world.create_dynamic_body(
    position=(-3, 2),
    fixtures=b2d.fixture_def(shape=b2d.circle_shape(radius=2), density=1),
)
right_wheel = world.create_dynamic_body(
    position=(3, 2),
    fixtures=b2d.fixture_def(shape=b2d.circle_shape(radius=2), density=1),
)

chasis = world.create_dynamic_body(
    position=(0, 2),
    fixtures=b2d.fixture_def(shape=b2d.polygon_shape(box=[3, 0.5]), density=1),
)

wheel_joint_def = dict(
    stiffness=10,
    enable_motor=True,
    motor_speed=-100,
    max_motor_torque=100,
    collide_connected=False,
    enable_limit=True,
    lower_translation=-0.4,
    upper_translation=0.4,
    local_axis_a=(0, 1),
)
world.create_wheel_joint(chasis, left_wheel, local_anchor_a=(-3, 0), **wheel_joint_def)
world.create_wheel_joint(chasis, right_wheel, local_anchor_a=(3, 0), **wheel_joint_def)


b2d.plot.animate_world(world, ppm=20, t=15, bounding_box=((-10,-5),(20,5)))

## Tutorial 3.6: Distance Joint

In [ ]:
world = b2d.world(gravity=(0, -10))

for i in range(10):

    # create static anchor (does not need shape/fixture)
    anchor = world.create_static_body(position=(i, 0))

    # 5 below the anchor
    body = world.create_dynamic_body(
        position=(i, -10),
        fixtures=b2d.fixture_def(shape=b2d.circle_shape(radius=0.4), density=0.5),
    )

    # distance joints of various stiffness-es
    world.create_distance_joint(anchor, body, length=10, stiffness=0.5 * (i + 1))

b2d.plot.animate_world(world, ppm=20, t=10, bounding_box=((-2,-20),(10,0)))

# Tutorial 4: Particles

In [ ]:
world = b2d.world(gravity=(0, -10))
pdef = b2d.particle_system_def(radius=0.1)
psystem = world.create_particle_system(pdef)

emitter_pos = (0, 0)
emitter_def = b2d.RandomizedLinearEmitterDef()
emitter_def.emite_rate = 400
emitter_def.lifetime = 5.1
emitter_def.size = (2, 1)
emitter_def.velocity = (6, 20)
emitter = b2d.RandomizedLinearEmitter(psystem, emitter_def)
b2d.plot.animate_world(world, ppm=20, t=10, bounding_box=((-10,-20),(20,5)), pre_step=emitter.step)